##RNN for sentiment analysis using IMDB dataset 

Preparing data

In [1]:
import torch #importing pytorch
from torchtext import data #pytorch library for preprocessing
import torch.nn.functional as F
#setting random seed
SEED = 1234
# setting manual seed using our random seed to get the same random number
torch.manual_seed(SEED)
# running on the CuDNN backend
torch.backends.cudnn.deterministic = True
# tokenizing text using spacy tokenizer
TEXT = data.Field(tokenize = 'spacy')
# Labelling our dataset and setting tensor to FloatTensors
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets # to get torch dataset
# downloading the IMDb dataset and spliting it into the canonical train/test splits as torchtext.datasets objects
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [3]:
# Checking length of train test splits
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['So', ',', 'neighbor', 'was', 'killing', 'neighbor', '.', 'Reminds', 'me', 'of', 'Iraq', '.', 'As', 'I', 'watched', 'the', 'American', 'flag', '(', '50', 'stars', 'in', '1864', '?', ')', 'being', 'dragged', 'behind', 'the', 'horse', ',', 'I', 'realized', 'why', 'burning', 'that', 'piece', 'of', 'red', 'white', 'and', 'blue', 'does', "n't", 'upset', 'me', 'as', 'much', 'as', 'our', 'destruction', '/', 'indifference', 'to', 'the', 'Bill', 'of', 'Rights', '.', 'I', "'m", 'a', 'Southerner', ',', 'and', 'must', 'have', 'some', 'historical', 'memory.<br', '/><br', '/>Watching', 'the', 'Tobey', 'McGuire', 'character', 'learn', 'to', 'respect', 'the', 'dignity', 'of', 'a', 'former', 'slave', ',', 'as', 'he', 'looks', 'at', 'the', 'scalps', 'of', 'blacks', 'and', 'Germans', '(', 'his', 'ethnic', 'background', ')', 'being', 'wagered', 'at', 'a', 'poker', 'game', '.....', 'was', 'interesting', '.', 'Many', 'twists', 'in', 'this', 'movie', '.', 'The', 'wife', ',', 'who', 'is', 'forced', 

In [5]:
import random
#creating valdation set and setting random_state so that we get training and validation set same each time
train_data, valid_data = train_data.split(random_state = random.seed(SEED),split_ratio=0.8)

In [6]:
# Checking length of train,test, validation splits
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [7]:
# building the vocabulary, only keeping the most common max_size tokens using our training set
MAX_VOCAB_SIZE = 25_000 

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
# Unique tokens in TEXT vocabulary and LABEL vocabulary
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
#to view the most common words in the vocabulary and their frequencies
print(TEXT.vocab.freqs.most_common(20))

[('the', 232582), (',', 221092), ('.', 189740), ('and', 125326), ('a', 124882), ('of', 114980), ('to', 107347), ('is', 87551), ('in', 70313), ('I', 62454), ('it', 61401), ('that', 56774), ('"', 50544), ("'s", 49584), ('this', 48507), ('-', 42205), ('/><br', 40788), ('was', 40044), ('as', 34706), ('with', 34223)]


In [10]:
# We can also see the vocabulary directly using either the stoi (string to int) or itos (int to string) method.
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fcda44b1ea0>, {'neg': 0, 'pos': 1})


In [12]:
# We then create the iterators. We iterate over these in the training/evaluation loop, and 
#they return a batch of examples (indexed and converted into tensors) at each iteration.
#We'll use a BucketIterator which is a special type of iterator that will return
#a batch of examples where each example is of a similar length, minimizing 
#the amount of padding per example.
#We also want to place the tensors returned by the iterator on the 
#GPU (if you're using one). PyTorch handles this using torch.device, 
#we then pass this device to the iterator.

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

Building our model

In [13]:
import torch.nn as nn # neural network library of pytorch
# Defining our RNN class as a sub-class of nn.Module 
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        """ this function takes input layer, embedding layer , hidden layer and output layer dimensions and creates layers of our NN"""
        #inheriting init function of nn.Module class 
        super().__init__()
        # embedding layer with input of input_dim and output of embedding_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        # hidden RNN layer with input of embedding_dim and output of hidden_dim
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        # ouput layer with input of input_dim and output of output_dim
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        # input batch passed through the embedding layer to get embedded
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        #feeding embedding into RNN
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        # verifying that hidden is simply the final hidden state where squeeze method is used to remove a dimension of size 1.
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        #feeding the last hidden state, hidden, through the linear layer, fc, to produce a prediction
        return self.fc(hidden.squeeze(0)).view(-1)

In [14]:
# to create an instance of our RNN class by passing INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM and OUTPUT_DIM
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [15]:
def count_parameters(model):
    """tell us how many trainable parameters our model has so we can compare the number of parameters across different models"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


Training our model

In [16]:
import torch.optim as optim #optimizer
# we use ADAM with learning rate as 0.0001
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [17]:
# BCEWithLogitsLoss criterion carries out both the sigmoid(since output of our RNN model is unbounded real number) and the binary cross entropy steps.
#criterion = nn.BCEWithLogitsLoss()

In [18]:
#putting model and criterion to our device
model = model.to(device)
#criterion = criterion.to(device)

In [19]:
def compute_binary_accuracy(model, data_loader, device):
    """ returns accuracy per batch i.e percentage of correct predictions to total number of examples"""
    model.eval()
    correct_pred, num_examples = 0, 0
    #turning off calculation of gradient
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            logits = model(batch_data.text)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100

In [20]:
import time 
start_time = time.time()
NUM_EPOCHS=15
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_iterator):
        
        ### FORWARD AND BACK PROP
        logits = model(batch_data.text)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_iterator):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_iterator, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_iterator, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_iterator, device):.2f}%')

Epoch: 001/015 | Batch 000/313 | Cost: 0.7252
Epoch: 001/015 | Batch 050/313 | Cost: 0.6887
Epoch: 001/015 | Batch 100/313 | Cost: 0.6962
Epoch: 001/015 | Batch 150/313 | Cost: 0.6910
Epoch: 001/015 | Batch 200/313 | Cost: 0.6946
Epoch: 001/015 | Batch 250/313 | Cost: 0.6927
Epoch: 001/015 | Batch 300/313 | Cost: 0.6927
training accuracy: 50.40%
valid accuracy: 49.90%
Time elapsed: 1.20 min
Epoch: 002/015 | Batch 000/313 | Cost: 0.6943
Epoch: 002/015 | Batch 050/313 | Cost: 0.6938
Epoch: 002/015 | Batch 100/313 | Cost: 0.6954
Epoch: 002/015 | Batch 150/313 | Cost: 0.7206
Epoch: 002/015 | Batch 200/313 | Cost: 0.6983
Epoch: 002/015 | Batch 250/313 | Cost: 0.6941
Epoch: 002/015 | Batch 300/313 | Cost: 0.6876
training accuracy: 50.37%
valid accuracy: 49.94%
Time elapsed: 2.39 min
Epoch: 003/015 | Batch 000/313 | Cost: 0.6914
Epoch: 003/015 | Batch 050/313 | Cost: 0.6916
Epoch: 003/015 | Batch 100/313 | Cost: 0.7147
Epoch: 003/015 | Batch 150/313 | Cost: 0.6923
Epoch: 003/015 | Batch 200/3

In [31]:
import spacy 
nlp = spacy.load('en')
def predict_sentiment(model, sentence):
    """ inputs the sentence and the model and outputs the sentiment of sentence based on the model"""
    model.eval()
    tokenized = [token.text for token in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor))
    if (prediction.item())>=0.5:
        return "Positive with score "+ str(prediction.item())
    else:
        return "Negative with score "+ str(prediction.item())
        

In [30]:
predict_sentiment(model, "I really love this movie.This movie is so great!")

'Positive with score 0.5495536923408508'